In [209]:
def get_attributes(filepath, folder):
    if not folder.endswith('/'):
        folder = folder + '/'

    file = folder + filepath
    file_stats = os.stat(file)
    attributes = {'parent_folder': folder,
                  'absolute_path': file,
                  'name': filepath,
                  'size_mb': file_stats.st_size / 1e+6,
                  'created_on': datetime.fromtimestamp(file_stats.st_birthtime),
                  'last_modified_on': datetime.fromtimestamp(file_stats.st_mtime)}

    if os.path.isdir(file):
        ext = 'folder'
    elif os.path.isfile(file):
        ext = filepath.split('.')[-1]
    else:
        ext = ''

    attributes['extension'] = ext

    return attributes


def blind_categories(format_dict, files):
    blind_classifier = {}

    for k, v in format_dict.items():
        blind_classifier[k] = [e for e in files if (e['extension'] in v)]

    blind_classifier['folders'] = [
        e for e in files if (e['extension'] == 'folder')]
    return blind_classifier


def extract_text(record, thresh=10000):
    text_ext = ['ppt', 'pptx', 'doc', 'docx', 'odt', 'pdf',
                'rtf', 'text', 'txt', 'wpd', 'ods', 'xls', 'xlsm', 'xlsx']

    if record['extension'] in text_ext:
        text = parser.from_file(record['absolute_path'], service='text')[
            'content'][:thresh]

    return text


def labels(corpus, nclusters=10):
    params = {
        'vectorizer': {
            'analyzer': 'word',
            'stop_words': stopwords.words('english'),
            'ngram_range': (1, 1),
            'token_pattern': '[a-z]{3,}',
            'max_df': 0.1,
            'lowercase': True
        },
        'km_params': {
            'random_state': 30,
            'n_clusters': nclusters,
        }
    }

    tfidf = TfidfVectorizer(**params['vectorizer'])
    try:
        X = tfidf.fit_transform(corpus)
        km = KMeans(**params['km_params']).fit(X)
        return km.labels_
    except:
        return 404


def name_cluster_labels_common(sw, topic_files, words=8):

    label_text = ' '.join([file['preproc_text'] for file in topic_files])
    corpus = re.findall('[a-zA-Z]{4,}', label_text)
    corpus = [w for w in corpus if (w.lower() not in sw)]
    label_dict = Counter(corpus)
    cluster_name = '-'.join([w[0] for w in label_dict.most_common(words)])

    return cluster_name


def get_topic_files(files):
    topic_files = defaultdict(list)

    for file in files:
        topic_files[file['topic']].append(file)

    topic_files = dict(topic_files)

    return topic_files



def preprocessing(text):
    text = text.replace('\n', ' ')
    text = re.sub('[#]\w+', ' ', text)
    for p in punctuation:
        text = text.replace(p, ' ')
    
    sent = []
    doc = nlp(text)
    
    for word in doc:
        sent.append(word.lemma_)
    
    return " ".join(sent).replace('-PRON-', '')


def stemming(text, all_stopwords=custom_sw):
    stemmer = SnowballStemmer('english', ignore_stopwords=True)
    stemmer.stopwords = all_stopwords
    
    stemmed_words = []
    
    for w in text.split():
        stemmed_words.append(stemmer.stem(w))
        
    text = ' '.join(stemmed_words)
        
    return text

In [168]:
from nltk.stem import SnowballStemmer
import spacy
import string
import re
from tika import parser
import os
import re
from datetime import datetime
from bs4 import BeautifulSoup
import chardet
import textract
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import Counter, defaultdict
from sklearn.utils.extmath import randomized_svd
import pickle
import os
import pymongo
from dotenv import load_dotenv
from datetime import datetime

nlp = spacy.load('en_core_web_lg')
punctuation = '!"$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
custom_sw = pickle.load(open('../app/data/stopwords.pkl', 'rb'))
custom_sw.extend(['want'])
formats = pickle.load(open('../app/data/formats.pkl', 'rb'))

# Access database

In [169]:
project_folder = os.path.expanduser(
    '/Users/faustina/METIS/BOOTCAMPWORK/Project5/controlledchaos/data/')  # the folder of your project
load_dotenv(os.path.join(project_folder, '.env'))

MONGODBNAME = os.getenv("MONGODBNAME")

client = pymongo.MongoClient()
db = client[MONGODBNAME]
curr_files_db = db['curr_files']

In [170]:
folder = '../data/test_case5/'
test_case = os.listdir(folder)

# Load files

In [171]:
curr_files_db.delete_many({})
curr_files_db.insert_many([get_attributes(file, folder) for file in test_case])
cursor = curr_files_db.find({})
entries = list(cursor)

# Categorization

In [172]:
files_w_attributes = entries

blind_cats = {k: list(curr_files_db.find({"extension": {"$in": v}})) for k, v in formats.items()}
blind_cats = {item: blind_cats[item] for item in sorted(blind_cats, key=lambda i: len(blind_cats[i]), reverse=True)}

In [173]:
def blind_categories(format_dict, collection):
    blind_classifier = {k: list(collection.find(
        {"extension": {"$in": v}})) for k, v in format_dict.items()}

    for k, v in blind_classifier.items():
        [(file.pop('_id', None) and file.pop('text', None)) for file in v]

    blind_classifier = {item: blind_classifier[item] for item in sorted(
        blind_classifier, key=lambda i: len(blind_classifier[i]), reverse=True)}
    return blind_classifier

# Text

1. Extraction
2. Preprocessing

    1. Remove punctuation marks
    2. Replace newline and tab characters with common spaces
    3. Lemmatize

3. Stemming (Snowball)
    1. Preserve stopwords
    2. Stem non-stopword words
    
4. Build corpus with lemmatized text
5. Convert corpus to a matrix of TF-IDF features
    1. Unigrams
    2. Words with more than 3 characters
    3. Terms with frequencies of 1% to 20%

In [175]:
for file in curr_files_db.find({'extension': {"$in": formats['text']}}):
    curr_files_db.update_one({'_id': file['_id']}, {"$set": {"text": extract_text(file)}})

In [176]:
for file in curr_files_db.find({'extension': {"$in": formats['text']}}):
    curr_files_db.update_one({'_id': file['_id'], "text": {"$exists": True}}, {"$set": {"preproc_text": preprocessing(file['text'])}})

In [193]:
for file in curr_files_db.find({'extension': {"$in": formats['text']}}):
    curr_files_db.update_one({'_id': file['_id'], "preproc_text": {"$exists": True}}, {"$set": {"lemma_text": stemming(file['preproc_text'])}})

In [195]:
corpus = [file['lemma_text'] for file in list(curr_files_db.find({'extension': {"$in": formats['text']}}))]

In [213]:
params = {
        'vectorizer': {
            'analyzer': 'word',
            'stop_words': stopwords.words('english'),
            'ngram_range': (1, 1),
            'token_pattern': '[a-z]{3,}',
            'min_df': 0.01,
            'max_df': 0.2,
            'lowercase': True
        }
    }

tfidf = TfidfVectorizer(**params['vectorizer'])

In [214]:
X_tfidf = tfidf.fit_transform(corpus)

# SVD

1. Calculate matrix decomposition to get sigma values
2. Calculate minimum optimal k to produce clusters

In [216]:
k = 100
U, Sigma, VT = randomized_svd(X_tfidf.toarray(), 
                              n_components=k,
                              n_iter=5,
                              random_state=None)

In [218]:
opt_k = 0

for idx, sig in enumerate(Sigma):
    if (idx + 1) < len(Sigma):
        if (str(sig)[:4] == str(Sigma[idx + 1])[:4]):
            print(str(sig)[:5], str(Sigma[idx + 1])[:5])
            print(idx)
            opt_k = idx
            break

1.966 1.963
39


# Clustering

In [220]:
entries = list(curr_files_db.find({"text": {"$exists": True}}))
raw_docs = [file['text'] for file in entries]
doc_labels = labels(raw_docs, nclusters=opt_k)

# Update database with labels

In [221]:
for idx, entry in enumerate(entries):
    curr_files_db.update_one({'_id': entry['_id']}, {"$set": {"label": 'topic{}'.format(doc_labels[idx])}})

# Agreggation

In [222]:
topic_files = {}

for i in range(20):
    entries = list(curr_files_db.find({"label": {"$exists": True}, "label": 'topic{}'.format(i)}))
    if len(entries):
        topic_name = name_cluster_labels_common(custom_sw, entries, words=5)
        topic_files[topic_name] = entries
    else:
        break

In [229]:
for k, v in topic_files.items():
    [[file.pop(file_key, None) for file_key in ['_id', 'text', 'preproc_text', 'lemma_text']] for file in v]

In [230]:
topic_files

{'boundary-need-feel-child-self': [{'parent_folder': '../data/test_case5/',
   'absolute_path': '../data/test_case5/2019-05-30_13-22-20_UTC.txt',
   'name': '2019-05-30_13-22-20_UTC.txt',
   'size_mb': 0.001154,
   'created_on': datetime.datetime(2020, 9, 14, 10, 25, 23),
   'last_modified_on': datetime.datetime(2020, 9, 14, 10, 25, 23),
   'extension': 'txt',
   'label': 'topic0'},
  {'parent_folder': '../data/test_case5/',
   'absolute_path': '../data/test_case5/2019-10-08_15-29-25_UTC.txt',
   'name': '2019-10-08_15-29-25_UTC.txt',
   'size_mb': 0.001014,
   'created_on': datetime.datetime(2020, 9, 14, 10, 25, 24),
   'last_modified_on': datetime.datetime(2020, 9, 14, 10, 25, 24),
   'extension': 'txt',
   'label': 'topic0'},
  {'parent_folder': '../data/test_case5/',
   'absolute_path': '../data/test_case5/2019-10-25_16-35-49_UTC.txt',
   'name': '2019-10-25_16-35-49_UTC.txt',
   'size_mb': 0.001608,
   'created_on': datetime.datetime(2020, 9, 14, 10, 25, 24),
   'last_modified_on'

In [231]:
pickle.dump(topic_files, open('../app/data/test5.pkl', 'wb'))